In [10]:
import requests
from bs4 import BeautifulSoup
import json

# URL của trang web cần lấy dữ liệu
url = "https://www.presight.io/privacy-policy.html"

# Gửi yêu cầu GET đến trang web
response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.content, 'html.parser')

privacy_content = []

# Biến lưu trạng thái hiện tại
current_header = None  
current_subheader = None  
header_classes = None  
subheader_classes = None  
is_after_a = False  
current_paragraph = None  

# Duyệt qua các thẻ cần thiết
for tag in soup.find_all(['h2', 'p', 'i', 'li', 'a']):
    tag_classes = tag.get("class", [])  

    if tag.name == 'h2':  # Xử lý tiêu đề chính (h2)
        is_after_a = False  
        current_paragraph = None 

        if current_subheader and tag_classes == subheader_classes:
            current_subheader = {
                "subheader": tag.get_text(strip=True),
                "class": tag_classes,
                "items": []
            }
            current_header["sub_content"].append(current_subheader)
        elif current_header and not current_header["sub_content"] and header_classes != tag_classes:
            current_subheader = {
                "subheader": tag.get_text(strip=True),
                "class": tag_classes,
                "items": []
            }
            current_header["sub_content"].append(current_subheader)
        else:  # Thêm một tiêu đề chính mới
            current_header = {
                "header": tag.get_text(strip=True),
                "class": tag_classes,
                "sub_content": []
            }
            header_classes = tag_classes
            privacy_content.append(current_header)
            current_subheader = None  # Đặt lại tiêu đề phụ

    elif tag.name == 'i' and current_header is not None:  # Xử lý tiêu đề phụ (i)
        is_after_a = False  
        current_paragraph = None  # Đặt lại đoạn văn hiện tại

        current_subheader = {
            "subheader": tag.get_text(strip=True),
            "class": tag_classes,
            "items": []
        }
        current_header["sub_content"].append(current_subheader)
        subheader_classes = tag_classes  # Cập nhật lớp CSS của tiêu đề phụ hiện tại

    elif tag.name == 'a':  # Xử lý thẻ liên kết (a)
        link_text = tag.get_text(strip=True)
        link_href = tag.get("href", "")
        privacy_content.append({
            "link_text": link_text,
            "href": link_href
        })
        is_after_a = True 
        current_paragraph = None  

    elif tag.name == 'p':  # Xử lý đoạn văn (p)
        content_text = tag.get_text(strip=True)
        current_paragraph = {
            "paragraph": content_text,
            "details": []
        }
        if is_after_a:
            privacy_content.append({
                "header": None,
                "sub_content": [content_text]
            })
        elif current_subheader is not None:
            current_subheader["items"].append(current_paragraph)
        elif current_header is not None:
            current_header["sub_content"].append(current_paragraph)
        else:
            privacy_content.append({
                "header": None,
                "sub_content": [content_text]
            })

    elif tag.name == 'li':  # Xử lý danh sách (li)
        content_text = tag.get_text(strip=True)
        if current_paragraph is not None:
            # Thêm nội dung vào "details" của đoạn văn gần nhất
            current_paragraph["details"].append(content_text)
        elif current_subheader is not None:
            current_subheader["items"].append(content_text)
        elif current_header is not None:
            current_header["sub_content"].append(content_text)

# Lưu nội dung vào tệp JSON
output_filename = "privacy_policy.json"
with open(output_filename, "w", encoding="utf-8") as file:
    json.dump(privacy_content, file, ensure_ascii=False, indent=4)

print(f"Crawl dữ liệu thành công! Dữ liệu đã được lưu vào tệp '{output_filename}'.")

Crawl dữ liệu thành công! Dữ liệu đã được lưu vào tệp 'privacy_policy.json'.
